In [ ]:
#the imports
import cv2
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import util
import time
#import that model:
import import_ipynb
import model

import os
#from argparse import ArgumentParser

In [ ]:
#The parameters
imbatch_read     = util.imbatch_read
img_write        = util.img_write
pickup_list      = util.pickup_list
ls_files_to_json = util.ls_files_to_json

build_generator     = model.build_generator

# args = build_parser().parse_args()

#-----------------TODO: LOAD YOU MODEL AND IMAGES HERE-------------
MODEL_SAVE_PATH = 'available_models/epch1/'
OUT_PATH = 'rendered_images/'
IMGSRC_PATH = 'test_images/'

if (not os.path.isdir(OUT_PATH)):
	os.makedirs(OUT_PATH)


BATCH_SIZE  = 1
FEED_SIZE   = 256
NOISE_RATE  = 0
#use_cpu, you can set this to True if GPU is not available
USE_CPU = False

DEVICE = ''
if USE_CPU =='TRUE':
	DEVICE = '/cpu:0'
    
input_ls = ls_files_to_json(IMGSRC_PATH, ext=['png', 'bmp', 'jpg', 'jpeg'])
CNT = len(input_ls)


In [ ]:
#start the render process

gpu_options = tf.GPUOptions(allow_growth=True)

with tf.device(DEVICE), tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
	input_r = tf.placeholder(tf.float32, shape=[BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3], name='inpr')
	g_state = build_generator(input_r, name='generator')
	g_var_ls = tf.trainable_variables(scope='generator')
	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver(g_var_ls)
	chkpt_fname = tf.train.latest_checkpoint(MODEL_SAVE_PATH)
	saver.restore(sess, chkpt_fname)
	# Warm up network and test...
	noise = np.random.normal(0, 1, size=(BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3)).astype(np.float32)
	sess.run(g_state, feed_dict={input_r: noise})
	# Begin...
	total = int(CNT / BATCH_SIZE) + (1 if (CNT % BATCH_SIZE) != 0 else 0)
	for offset in range(total):
		sub_ls  = pickup_list(input_ls, BATCH_SIZE, offset * BATCH_SIZE)
		sub_img, sls = imbatch_read(IMGSRC_PATH, sub_ls, (FEED_SIZE, FEED_SIZE))
		if NOISE_RATE != 0:
			noise = np.random.normal(0, 1,
				size=(BATCH_SIZE, FEED_SIZE, FEED_SIZE, 3)).astype(np.float32)
			sub_img = sub_img + noise * NOISE_RATE 
		time_start = time.time()
		render_batch = sess.run(g_state, feed_dict={input_r: sub_img})
		print('Processing: %d/%d, network dataflow time: %f'%(offset + 1, total, time.time()-time_start))
		for i in range(len(sls)):
			img_write(render_batch[i],
					OUT_PATH+'/%d_%d.jpg'%(offset, i),
				sls[i])# (FEED_SIZE, FEED_SIZE)) #